In [5]:
import requests
from bs4 import BeautifulSoup
import re
import string
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
def retrieve_docs_and_clean():
    r = requests.get('https://sports.ndtv.com/fifa-world-cup-2022/news')
    soup = BeautifulSoup(r.content, 'html.parser')
    #THE FOLLOWING CODE NEED TO BE MODIFIED TO SUITE FOR THE ABOVE URL
    link = []
    for i in soup.find('div', {'class':'lst-pg_hd'}).find_all('a',{'class':'lst-pg_ttl'}):
        i['href'] ='https://sports.ndtv.com/'+ i['href'] + '?page=all'
        link.append(i['href'])
    # Retrieve Paragraphs
    documents = []
    for i in link:
        r = requests.get(i)
        soup = BeautifulSoup(r.content, 'html.parser')
        sen = []
        for i in soup.find_all('p'):
            sen.append(i.text)
        documents.append(' '.join(sen))
    # Clean Paragraphs
    documents_clean = []
    for d in documents:
        document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
        document_test = re.sub(r'@\w+', '', document_test)
        document_test = document_test.lower()
        document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
        document_test = re.sub(r'[0-9]', '', document_test)
        document_test = re.sub(r'\s{2,}', ' ', document_test)
        documents_clean.append(document_test)
    return documents_clean    

In [6]:
docs = retrieve_docs_and_clean()

# Create Term-Document Matrix with TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
print(df.head())
print(df.shape)

                  0         1         2    3         4         5    6   \
aboukhlal   0.000000  0.000000  0.000000  0.0  0.000000  0.000000  0.0   
about       0.026409  0.002785  0.000000  0.0  0.009913  0.005084  0.0   
above       0.000000  0.000000  0.017942  0.0  0.000000  0.000000  0.0   
academy     0.000000  0.000000  0.000000  0.0  0.000000  0.000000  0.0   
accumulate  0.000000  0.000000  0.000000  0.0  0.000000  0.000000  0.0   

                  7         8         9    10        11   12   13        14  \
aboukhlal   0.000000  0.000000  0.000000  0.0  0.060186  0.0  0.0  0.000000   
about       0.000000  0.008038  0.025978  0.0  0.022795  0.0  0.0  0.013897   
above       0.000000  0.055728  0.000000  0.0  0.000000  0.0  0.0  0.000000   
academy     0.057842  0.000000  0.000000  0.0  0.000000  0.0  0.0  0.000000   
accumulate  0.000000  0.063668  0.000000  0.0  0.000000  0.0  0.0  0.000000   

             15   16        17  
aboukhlal   0.0  0.0  0.000000  
about       0.

In [7]:
docs = retrieve_docs_and_clean()
# Create Term-Document Matrix with TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
aboukhlal,0.000000,0.00000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.00000,0.0,0.060097,0.0,0.0,0.000000,0.0,0.0,0.000000
about,0.025812,0.00278,0.000000,0.0,0.009823,0.00505,0.0,0.000000,0.007963,0.02553,0.0,0.022761,0.0,0.0,0.014448,0.0,0.0,0.007967
above,0.000000,0.00000,0.017386,0.0,0.000000,0.00000,0.0,0.000000,0.055208,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
academy,0.000000,0.00000,0.000000,0.0,0.000000,0.00000,0.0,0.059203,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
accumulate,0.000000,0.00000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.063074,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


In [8]:
def get_similar_articles(q, df):
    print("query:", q)
    print("The following are articles with the highest cosine similarity values: ")
    q = [q]
    q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    sim = {}
    for i in range(10):
        sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
    sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  
    for k, v in sim_sorted:
        if v != 0.0:
            print("Similarity Values:", v)
            print(docs[k])
            print()

In [9]:
q1 = 'barcelona'
q2 = 'spain'
q3 = 'argentina'

get_similar_articles(q1, df)
print('-'*100)
get_similar_articles(q2, df)
print('-'*100)
get_similar_articles(q3, df)

query: barcelona
The following are articles with the highest cosine similarity values: 
----------------------------------------------------------------------------------------------------
query: spain
The following are articles with the highest cosine similarity values: 
Similarity Values: 0.04918072870534763
following his side s loss to morocco in the quarterfinal of the ongoing fifa world cup portugal coach fernando santos said that he had no regrets about not including star striker cristiano ronaldo in starting lineup morocco scripted history on saturday to become the first african nation to qualify for the last four of the world cup after defeating portugal in the quarterfinal our players are distressed said portugal coach fernando santos as quoted by sky sports ronaldo had come to the field after fifty minutes but could not make a difference and left the pitch teary eyed as the world cup dreams of this year old were broken in pieces santos added cristiano is a great player and he

In [10]:
pip install rank_bm25

Note: you may need to restart the kernel to use updated packages.


In [11]:
from rank_bm25 import BM25Okapi

In [12]:
tokenized_corpus = [doc.split(" ") for doc in docs]
bm25 = BM25Okapi(tokenized_corpus)
def rank_bm25(docs,query):
    print("query:", query)
    print("The following are articles with the highest cosine similarity values: ")
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    rang={}
    for i in range (len(doc_scores)):
        if doc_scores[i]!=0.:
            rang[i]=doc_scores[i]
    rang_sorted = sorted(rang.items(), key=lambda x: x[1], reverse=True)
    for i,j in rang_sorted[:3]:
        print("Similarity Values:", j)
        print(docs[i])
        print()
    print('-'*100)

In [13]:
q1 = 'barcelona'
q2 = 'spain'
q3 = 'argentina'
rank_bm25(docs,q1)
rank_bm25(docs,q2)
rank_bm25(docs,q3)

query: barcelona
The following are articles with the highest cosine similarity values: 
----------------------------------------------------------------------------------------------------
query: spain
The following are articles with the highest cosine similarity values: 
Similarity Values: 1.5101566040067051
cristiano ronaldo has equalled the world record for international caps of after coming on as a substitute in the st minute in the fifa world cup quarter final against morocco on saturday earlier cristiano ronaldo started on the bench once more in portugal s world cup quarter final clash against morocco on saturday coach fernando santos dropped the five time ballon d or winner for the last win over switzerland and maintained a similar line up against the north africans goncalo ramos who scored a hat trick in the victory leads the line again with support from bruno fernandes bernardo silva and joao felix ruben neves replaces william carvalho in defensive midfield for santos s only c